In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.087, 0.084, 0.115, 0.111],
    [0.066, 0.079, 0.111, 0.102],
    [0.044, 0.086, 0.085, 0.088],
    [0.088, 0.086, 0.244, 0.078],
    [0.285, 0.063, 0.122, 0.068],
    [0.040, 0.134, 0.018, 0.144],
    [0.030, 0.096, 0.020, 0.148],
    [0.039, 0.130, 0.112, 0.105],
    [0.109, 0.103, 0.124, 0.089], 
    [0.014, 0.139, 0.040, 0.069]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.15288888, 0.30888675, 0.45773133, 0.080493041])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site4   0.088  0.086  0.244  0.078      0.397820       0.065269  0.332551   
Site8   0.039  0.130  0.112  0.105      0.184842       0.061009  0.123833   
Site9   0.109  0.103  0.124  0.089      0.156112       0.072289  0.083823   
Site1   0.087  0.084  0.115  0.111      0.106592       0.106562  0.000029   
Site5   0.285  0.063  0.122  0.068      0.144276       0.160689 -0.016412   
Site2   0.066  0.079  0.111  0.102      0.092518       0.126588 -0.034070   
Site10  0.014  0.139  0.040  0.069      0.134320       0.196842 -0.062522   
Site3   0.044  0.086  0.085  0.088      0.063392       0.158806 -0.095414   
Site6   0.040  0.134  0.018  0.144      0.121379       0.229851 -0.108472   
Site7   0.030  0.096  0.020  0.148      0.040112       0.263458 -0.223346   

        Rank  
Site4      1  
Site8      2  
Site9      3  
Site1      4  
Site5      5  
Site2      6  
Site10     7  
Site3      8  
Site6      9  
Si